In [2]:
import numpy as np 
import torch
import torchvision.models as models
import torchvision.datasets as dsets
from torch.utils.data import DataLoader 


from tqdm import tqdm 


torch.backends.cuda.matmul.allow_tf32 = False
torch.backends.cuda.matmul.allow_fp16_reduced_precision_reduction = False
torch.backends.cudnn.enabled = True
torch.backends.cudnn.allow_tf32 = False
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1).to('cuda')
# model = models.vgg19(weights=models.VGG19_Weights.IMAGENET1K_V1).to('cuda')
# model = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.IMAGENET1K_V1).to('cuda')
# model = models.efficientnet_b2(weights=models.EfficientNet_B2_Weights.IMAGENET1K_V1).to('cuda')
model = models.efficientnet_b2(weights=models.EfficientNet_B2_Weights.IMAGENET1K_V1)
# FP16
model.half()

for name, param in model.named_parameters():
     Data_shape = param.shape
     #FP16, FP32
     #print(param.view(torch.uint8).view(-1))
     
     Data_1d = param.view(-1)
     Data_1d_int = Data_1d.view(torch.uint8) 
     
          
     # FP16
     #Data_1d_int [::2] = 0   # 8bit - uint8 : 9.744    68.776
     # print(param.view(torch.uint8).view(-1))
     #                                       RESNET
     # Data_1d_int[::2] &= 0x80        # 7bit - 62.592
     # Data_1d_int[::2] &= 0xC0        # 6bit - 73.212
     # Data_1d_int[::2] &= 0xE0        # 5bit - 73.37
     # Data_1d_int[::2] &= 0xF0       # 4bit - 75.92 
     # Data_1d_int[::2] &= 0xF8       # 3bit - 76.026
     # Data_1d_int[::2] &= 0xFC        # 2bit - 76.0660
     # Data_1d_int[::2] &= 0xFE         # 1bit - 76.112
     
     
     # FP32
     # Data_1d_int [::4] = 0
     # Data_1d_int [1::4] = 0
     # print(param.view(torch.uint8).view(-1))
     
# FP16
model.float() 
          
# dataset = dsets.ImageFolder("/media/imagenet/val", models.ResNet50_Weights.IMAGENET1K_V1.transforms()) ### 2번째 인자, transform
#dataset = dsets.ImageFolder("/media/imagenet/val", models.VGG19_Weights.IMAGENET1K_V1.transforms()) ### 2번째 인자, transform
# dataset = dsets.ImageFolder("/media/imagenet/val", models.MobileNet_V2_Weights.IMAGENET1K_V1.transforms()) ### 2번째 인자, transform
dataset = dsets.ImageFolder("/media/imagenet/val", models.EfficientNet_B2_Weights.IMAGENET1K_V1.transforms()) ### 2번째 인자, transform

loader = DataLoader(dataset= dataset, # dataset
                   batch_size=64,   # batch size power to 2
                   shuffle = False, # false
                   num_workers = 8, # num_workers 
                   pin_memory=True) # pin_memory 

correct = 0
total = 50000

model.eval()

for input, label in tqdm(loader):
    input = input#.cuda(non_blocking=True)
    label = label#.cuda(non_blocking=True)
    output = model(input)
    pred = torch.argmax(output, 1)
    correct += (pred == label).int().sum()
acc1 = correct / total * 100

print(acc1)

100%|██████████| 782/782 [22:11<00:00,  1.70s/it]

tensor(80.6100)
